In [ ]:
# Installat

# Imports and Setup

In [ ]:
# Imports and Setup
import tkinter as tk
from tkinter import messagebox
import speech_recognition as sr
from googletrans import Translator
import noisereduce
import numpy as np


# User Interface Units

In [ ]:

class DynamicUI:
    def __init__(self, root, translation_app, role):
        self.root = root
        self.translation_app = translation_app
        self.role = role
        self.create_ui()

    def create_ui(self):
        self.message_label = tk.Label(self.root, text=f"{self.role.capitalize()}: Speak in {'English' if self.role == 'sender' else 'Hindi'}:")
        self.message_label.pack()

        self.message_entry = tk.Entry(self.root)
        self.message_entry.pack()

        self.recognize_button = tk.Button(self.root, text="Recognize", command=self.recognize_speech)
        self.recognize_button.pack()

    def recognize_speech(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            try:
                audio = recognizer.listen(source, timeout=5)

                # Apply noise reduction
                audio_data = np.frombuffer(audio.frame_data, dtype=np.int16)
                sr_value = audio.sample_rate  # Get sample rate
                reduced_noise = noisereduce.reduce_noise(audio_data, sr_value)
                reduced_audio = sr.AudioData(reduced_noise.tobytes(), audio.sample_rate, audio.sample_width)

                message = recognizer.recognize_google(reduced_audio, language="en" if self.role == 'sender' else "hi-IN")
                self.message_entry.delete(0, tk.END)
                self.message_entry.insert(0, message)
                messagebox.showinfo(f"{self.role.capitalize()} Message", f"{self.role.capitalize()}: {message}")

                if self.role == 'sender':
                    translated_message = self.translate_message(message, source_lang="en", target_lang="hi")
                    self.translation_app.receive_translated_message(self.role, translated_message)
                else:
                    self.translation_app.receive_translated_response(self.role, message)

            except sr.UnknownValueError:
                messagebox.showwarning("Recognition Error", "Could not understand audio.")
            except sr.RequestError as e:
                messagebox.showerror("Recognition Error", f"Error with the speech recognition service; {e}")

    def translate_message(self, message, source_lang, target_lang):
        translator = Translator()
        translated_message = translator.translate(message, src=source_lang, dest=target_lang).text
        return translated_message

class TranslationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Dynamic Translation App")

        # Create Sender UI
        sender_frame = tk.Frame(root)
        self.sender_ui = DynamicUI(sender_frame, self, role='sender')
        sender_frame.pack(side=tk.LEFT)

        # Create Receiver UI
        receiver_frame = tk.Frame(root)
        self.receiver_ui = DynamicUI(receiver_frame, self, role='receiver')
        receiver_frame.pack(side=tk.RIGHT)

    def receive_translated_message(self, role, translated_message):
        if role == 'sender':
            self.receiver_ui.message_entry.delete(0, tk.END)
            self.receiver_ui.message_entry.insert(0, translated_message)
            messagebox.showinfo("Receiver Message", f"Receiver: {translated_message}")
        else:
            self.sender_ui.message_entry.delete(0, tk.END)
            self.sender_ui.message_entry.insert(0, translated_message)
            messagebox.showinfo("Sender Message", f"Sender: {translated_message}")

    def receive_translated_response(self, role, translated_response):
        if role == 'receiver':
            translated_message = self.translate_message(translated_response, source_lang="hi", target_lang="en")
            self.sender_ui.message_entry.delete(0, tk.END)
            self.sender_ui.message_entry.insert(0, translated_message)
            messagebox.showinfo("Sender Message", f"Sender: {translated_message}")
        else:
            self.receiver_ui.message_entry.delete(0, tk.END)
            self.receiver_ui.message_entry.insert(0, translated_response)
            messagebox.showinfo("Receiver Message", f"Receiver: {translated_response}")

    def translate_message(self, message, source_lang, target_lang):
        translator = Translator()
        translated_message = translator.translate(message, src=source_lang, dest=target_lang).text
        return translated_message


# Main Application Loop

In [ ]:

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslationApp(root)
    root.mainloop()